### Compute viscosity from json file

This cookbook is a calculator of rheology

First, we load a rheology from file or a dict

Then we pass T, P, strain rate condition and compute the viscosities.

In [ ]:
import os
import json
import os
import sys
import numpy as np
from matplotlib import pyplot as plt

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.Rheology as Rheology

#### Step 1: Take the original WarrenHansen23 and compute the rheology at the reference state

In [ ]:
# import the _dict directly

# version 1

WarrenHansen23_disl =\
            {
                "A": 20,
                "p": 0.0,
                "r": 1.2,
                "n": 3.5,
                "E": 480e3,
                "V": 11e-6
            }

WarrenHansen23_diff = \
            {
                "A" : 2.9e5,
                "p" : 3.0,
                "r" : 1.0,
                "n" : 1.0,
                "E" : 335e3,
                "V" : 4e-6
            }

Pref = 100.0e6 # Pa
Tref = 1250.0 + 273.15 # K
stress_ref = 50 # MPa
dref = 15.0  # mum
Coh_ref = 300.0 # H / 10^6 Si

strain_rate_WH23_diff = Rheology.CreepStrainRate(WarrenHansen23_diff, stress_ref, Pref, Tref, dref, Coh_ref)
strain_rate_WH23_disl = Rheology.CreepStrainRate(WarrenHansen23_disl, stress_ref, Pref, Tref, dref, Coh_ref)

eta_WH23_diff = Rheology.CreepRheology(WarrenHansen23_diff, strain_rate_WH23_diff, Pref, Tref, dref, Coh_ref, use_effective_strain_rate=False)
eta_WH23_disl = Rheology.CreepRheology(WarrenHansen23_disl, strain_rate_WH23_disl, Pref, Tref, dref, Coh_ref, use_effective_strain_rate=False)

print("strain_rate_WH23_diff: ", strain_rate_WH23_diff)
print("stress_WH23_diff: ", strain_rate_WH23_diff * 2.0 * eta_WH23_diff)
print("strain_rate_WH23_disl: ", strain_rate_WH23_disl)
print("stress_WH23_disl: ", strain_rate_WH23_disl * 2.0 * eta_WH23_disl)

In [ ]:
def CreepRheology(creep, strain_rate, P, T, d=1e4, Coh=1e3, **kwargs):
    """
    Calculate viscosity by flow law in form of (strain_rate)**(1.0 / n - 1) * (B)**(-1.0 / n) * np.exp((E + P * V) / (n * R * T))
    Previously, there is a typo in the F factor
    Units:
     - P: Pa
     - T: K
     - d: mu m
     - Coh: H / 10^6 Si
     - Return value: Pa*s
    Pay attention to pass in the right value, this custom is inherited
    """

    R = 8.314
    
    A = creep['A']
    p = creep['p']
    r = creep['r']
    n = creep['n']
    E = creep['E']
    V = creep['V']
    # compute value of F(pre factor)
    use_effective_strain_rate = kwargs.get('use_effective_strain_rate', False)
    f_by_factor = kwargs.get('f_by_factor', False)
    if use_effective_strain_rate:
        F = 1 / (2**((n-1)/n)*3**((n+1)/2/n)) * 2.0
    elif f_by_factor:
        F = kwargs['F']
    else:
        F = 1.0
    # calculate B
    B = A * d**(-p) * Coh**r
    eta = 1/2.0 * F * (strain_rate)**(1.0 / n - 1) * (B)**(-1.0 / n) * np.exp((E + P * V) / (n * R * T)) * 1e6

    return eta

In [ ]:
strain_coeff = 2.0 / (3**0.5)
stress_coeff = 3**0.5

eta_WH23_diff = CreepRheology(WarrenHansen23_diff, strain_rate_WH23_diff/ strain_coeff, Pref, Tref, dref, Coh_ref, use_effective_strain_rate=True)
eta_WH23_disl = CreepRheology(WarrenHansen23_disl, strain_rate_WH23_disl/ strain_coeff, Pref, Tref, dref, Coh_ref, use_effective_strain_rate=True)

print("strain_rate_WH23_diff: ", strain_rate_WH23_diff)
print("stress_WH23_diff: ", strain_rate_WH23_diff /strain_coeff * 2.0 * eta_WH23_diff * stress_coeff)
print("strain_rate_WH23_disl: ", strain_rate_WH23_disl)
print("stress_WH23_disl: ", strain_rate_WH23_disl /strain_coeff * 2.0 * eta_WH23_disl * stress_coeff)

In [ ]:
eta_WH23_diff = CreepRheology(WarrenHansen23_diff, strain_rate_WH23_diff, Pref, Tref, dref, Coh_ref)
eta_WH23_disl = CreepRheology(WarrenHansen23_disl, strain_rate_WH23_disl, Pref, Tref, dref, Coh_ref)

print("strain_rate_WH23_diff: ", strain_rate_WH23_diff)
print("stress_WH23_diff: ", strain_rate_WH23_diff * 2.0 * eta_WH23_diff)
print("strain_rate_WH23_disl: ", strain_rate_WH23_disl)
print("stress_WH23_disl: ", strain_rate_WH23_disl * 2.0 * eta_WH23_disl)

#### Step 2: enter the rheology used in ASPECT

In [ ]:
# _dict = {'diffusion_creep': {'A': 2.5670749211437165e-16, 'd': 0.01, 'n': 1.0, 'm': 3.0, 'E': 335000.0, 'V': 1.9e-06},
# 'dislocation_creep': {'A': 1.30864197280044e-15, 'd': 0.01, 'n': 3.5, 'm': 0.0, 'E': 480000.0, 'V': 1.6e-05},
# 'diffusion_lm': {'A': 1.227183423149255e-17, 'd': 0.01, 'n': 1.0, 'm': 3.0, 'E': 335000.0, 'V': 3e-06}}

_dict = {'diffusion_creep': {'A': 1.2835374605718582e-16, 'd': 0.01, 'n': 1.0, 'm': 3.0, 'E': 335000.0, 'V': 1.9e-06}, 
'dislocation_creep': {'A': 1.1566870163906657e-16, 'd': 0.01, 'n': 3.5, 'm': 0.0, 'E': 480000.0, 'V': 1.6e-05}, 
'diffusion_lm': {'A': 3.0679585578731375e-18, 'd': 0.01, 'n': 1.0, 'm': 3.0, 'E': 335000.0, 'V': 3e-06}}

diffusion_creep = _dict["diffusion_creep"]
dislocation_creep = _dict["dislocation_creep"]

#### Step 3: compute the stress at the reference state and compare with the reference value

In [ ]:
strain_coeff = 2.0 / 3**0.5
stress_coeff = 3**0.5

In [ ]:
diffusion_creep['d'] = dref / 1e6 # m

diffusion_creep_eta = Rheology.CreepRheologyInAspectViscoPlastic(diffusion_creep, strain_rate_WH23_diff / strain_coeff, Pref, Tref)
diffusion_creep_stress = diffusion_creep_eta * 2 * strain_rate_WH23_diff / strain_coeff * stress_coeff

print("diffusion creep viscosity: ", diffusion_creep_eta)
print("diffusion creep stress: ", diffusion_creep_stress)

In [ ]:
dislocation_creep['d'] = dref / 1e6 # m

dislocation_creep_eta = Rheology.CreepRheologyInAspectViscoPlastic(dislocation_creep, strain_rate_WH23_disl/ strain_coeff, Pref, Tref)
dislocation_creep_stress = dislocation_creep_eta * 2 * strain_rate_WH23_disl / strain_coeff * stress_coeff


print("dislocation creep viscosity: ", dislocation_creep_eta)
print("dislocation creep stress: ", dislocation_creep_stress)